# Assignment 6.2 - More Trees

Please submit your solution of this notebook in the Whiteboard at the corresponding Assignment entry as .ipynb-file and as .pdf. <br><br>
Please do **NOT** rename the file!

#### State both names of your group members here:
[Jane and John Doe]

In [1]:
# Paola Gega, Daniel Thompson

---

## Grading Info/Details - Assignment 6.2:

The assignment will be graded semi-automatically, which means that your code will be tested against a set of predefined test cases and qualitatively assessed by a human. This will speed up the grading process for us.

* For passing the test scripts: 
    - Please make sure to **NOT** alter predefined class or function names, as this would lead to failing of the test scripts.
    - Please do **NOT** rename the files before uploading to the Whiteboard!

* **(RESULT)** tags indicate checkpoints that will be specifically assessed by a human.

* You will pass the assignment if you pass the majority of test cases and we can at least confirm effort regarding the **(RESULT)**-tagged checkpoints per task.

---

## Task 6.2.1 - Classification Trees

* Implement the Classification Tree Class from scratch using only `NumPy`. The splitting criterion should be Gini impurity. **(RESULT)**
* Run your implementation on the `IRIS` classification dataset. **(RESULT)**

In [2]:
import numpy as np
from sklearn.datasets import load_iris

class DecisionTree:
    """Base class for a decision tree."""

    def __init__(self, max_depth=5, min_samples_split=10):
        """Initialize decision tree."""
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = {}
    
    def fit(self, X, y):
        """Build the decision tree."""
        self.tree = self._build_tree(X, y, 0)

    def _loss(self, y1, y2):
        """This method is designed to be overridden by subclasses."""
        pass

    def _get_split_candidates(self, feature_values):
        """
        Determines the split candidates for a given feature.
        This method is designed to be overridden by subclasses.
        By default, it returns unique values, but subclasses like ExtraTree
        can implement a different strategy (e.g., random thresholds).
        """
        return np.unique(feature_values)

    def _build_tree(self, X, y, depth):
        """Recursively build the tree."""
        n_samples, n_features = X.shape
        # Make node a leaf if stop conditions are met
        if len(np.unique(y)) == 1:
            return {'value': y[0]}
        elif (depth >= self.max_depth or n_samples < self.min_samples_split):
            values, counts = np.unique(y, return_counts=True)
            ind = np.argmax(counts)
            return {'value': values[ind]}
        # Search for optimal split
        best_j = None
        best_z = None
        best_loss = float('inf')
        for j in range(n_features):
            X_j = self._get_split_candidates(X[:,j])
            for z in (X_j[:-1]+X_j[1:])/2:
                # Calculate impurity of split
                left_indices = X[:, j] <= z
                right_indices = ~left_indices
                loss = self._loss(y[left_indices], y[right_indices])
                if loss < best_loss:
                    best_loss = loss
                    best_j = j
                    best_z = z
        left_indices = X[:, best_j] <= best_z
        right_indices = ~left_indices
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)
        return {'feature_index' : best_j, 
                'threshold' : best_z,
                'left' : left_subtree,
                'right' : right_subtree}
    
    def predict(self, X):
        """Make predictions for X."""
        n_samples = X.shape[0]
        y_pred = np.empty(n_samples, dtype=float)
        for i in range(n_samples):
            node = self.tree
            while 'value' not in node.keys():
                if X[i,node['feature_index']] <= node['threshold']:
                    node = node['left']
                else:
                    node = node['right']
            y_pred[i] = node['value']
        return y_pred

In [3]:
def gini(y):
            counts = np.unique(y, return_counts=True)[1]
            prob_sq_sum = sum((count / len(y)) ** 2 for count in counts)
            return 1 - prob_sq_sum

class ClassificationTree(DecisionTree):
    """Classification decision tree using Gini impurity."""
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def _loss(self, y1, y2):
        """Calculate Gini impurity of a split."""
        total_samples = len(y1) + len(y2)
        gini1 = gini(y1)
        gini2 = gini(y2)
        weighted_gini = (len(y1) * gini1 + len(y2) * gini2) / total_samples
        return weighted_gini

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_wine

data = load_wine()
X = data.data
y = data.target

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
# Train a classification tree model where every leaf corresponds to a single sample
model = ClassificationTree(max_depth = 150, min_samples_split = 2)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("Accuracy on training set:", accuracy_score(y_train, y_train_pred))
y_test_pred = model.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_test_pred))

Accuracy on training set: 1.0
Accuracy on test set: 0.9333333333333333


In [6]:
# Train a classification tree model with more reasonable parameters to try to get less overfitting
model = ClassificationTree(max_depth = 5, min_samples_split = 20)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("Accuracy on training set:", accuracy_score(y_train, y_train_pred))
y_test_pred = model.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_test_pred))

Accuracy on training set: 0.9924812030075187
Accuracy on test set: 0.9333333333333333


## Task 6.2.2 - Random Forests

* Implement Random Forests using only `NumPy`. **(RESULT)**
* Compare the results between the random forest run of your `ClassificationTree` class on the `IRIS` dataset. **(RESULT)**

In [7]:
class RandomForest:
    """Random Forest Classifier."""
    def __init__(self, bags, bag_size=None, max_depth=5, min_samples_split=10, random_state=None):
        self.bags = bags
        self.bag_size = bag_size
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.trees = []
    
    def fit(self, X, y):
        """Fit the random forest."""
        n_samples, n_features = X.shape
        if self.bag_size is None:
            self.bag_size = n_samples
        n = np.floor(np.sqrt(n_features)).astype(int)
        np.random.seed(self.random_state)
        for _ in range(self.bags):
            sample_indices = np.random.choice(n_samples, self.bag_size, replace=True)
            feature_indices = np.random.choice(n_features, n, replace=False)
            X_sample, y_sample = X[sample_indices][:,feature_indices], y[sample_indices]

            tree = ClassificationTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_sample, y_sample)
            self.trees.append((tree, feature_indices))
    
    def predict(self, X):
        """Make predictions by majority voting."""
        # Collect predictions from each tree
        predictions = np.empty((X.shape[0], self.bags), dtype=int)

        for i, (tree, feature_indices) in enumerate(self.trees):
            # Select the same features as used in fitting
            X_sample = X[:, feature_indices]
            predictions[:, i] = tree.predict(X_sample)

        return np.array([np.bincount(predictions[i]).argmax() for i in range(predictions.shape[0])])

In [8]:
# Train a random forest model with the same parameters as before
model = RandomForest(bags = 10, max_depth = 5, min_samples_split = 20, random_state=42)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print("Accuracy on training set:", accuracy_score(y_train, y_train_pred))
y_test_pred = model.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_test_pred))

Accuracy on training set: 0.9924812030075187
Accuracy on test set: 1.0


## Task 6.2.3 - Extra Trees

* Implement Extra Trees using only `NumPy`. **(RESULT)**
* Compare the results between the `Random Forest` and an `Extra Trees` ensemble implementation on the `IRIS` dataset. **(RESULT)**

In [9]:
class ExtraTree(ClassificationTree):
    """Extremely Randomized Tree - uses random thresholds instead of optimal ones."""
    def __init__(self, random_state, *args, **kwargs):
        self.random_state = random_state
        super().__init__(*args, **kwargs)

    def _get_split_candidates(self, feature_values):
        """
        Determines the split candidates for a given feature.
        """
        np.random.seed(self.random_state)
        unique_feature_values = np.unique(feature_values)
        n_values = len(unique_feature_values)
        # num_thresholds = np.random.randint(n_values)+1
        num_thresholds = np.floor(np.sqrt(n_values)).astype(int)
        indices = np.random.choice(n_values, num_thresholds, replace=False)
        return unique_feature_values[indices]

In [10]:
class ExtraTrees:
    """Extremely Randomized Trees ensemble."""
    def __init__(self, bags, bag_size=None, max_depth=5, min_samples_split=10, random_state=None):
        self.bags = bags
        self.bag_size = bag_size
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.random_state = random_state
        self.trees = []
    
    def fit(self, X, y):
        """Fit the random forest."""
        n_samples, n_features = X.shape
        if self.bag_size is None:
            self.bag_size = n_samples
        np.random.seed(self.random_state)
        for _ in range(self.bags):
            sample_indices = np.random.choice(n_samples, self.bag_size, replace=True)
            X_sample, y_sample = X[sample_indices], y[sample_indices]

            tree = ExtraTree(max_depth=self.max_depth, min_samples_split=self.min_samples_split, random_state=self.random_state)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)
    
    def predict(self, X):
        """Make predictions by averaging all trees."""
        # Collect predictions from each tree
        predictions = np.empty((X.shape[0], self.bags), dtype=int)

        for i, tree in enumerate(self.trees):
            predictions[:, i] = tree.predict(X)

        # Majority voting
        return np.array([np.bincount(predictions[i]).argmax() for i in range(predictions.shape[0])])

In [12]:
# Create and train the Extra Tree
extra_tree = ExtraTree(max_depth=5, min_samples_split=5, random_state=42)
extra_tree.fit(X_train, y_train)
y_train_pred = extra_tree.predict(X_train)
y_test_pred = extra_tree.predict(X_test)
print("Extra tree")
print("Accuracy on training set:", accuracy_score(y_train, y_train_pred))
print("Accuracy on test set:", accuracy_score(y_test, y_test_pred))
print()

# Create and train the Extra Trees ensemble
extra_trees = ExtraTrees(bags=100, max_depth=5, min_samples_split=5, random_state=42)
extra_trees.fit(X_train, y_train)
y_train_pred = extra_trees.predict(X_train)
y_test_pred = extra_trees.predict(X_test)
print("Extra trees ensemble")
print("Accuracy on training set:", accuracy_score(y_train, y_train_pred))
print("Accuracy on test set:", accuracy_score(y_test, y_test_pred))

Extra tree
Accuracy on training set: 0.9849624060150376
Accuracy on test set: 0.9333333333333333

Extra trees ensemble
Accuracy on training set: 0.9097744360902256
Accuracy on test set: 0.8666666666666667


## Congratz, you made it! :)